# PCA y Manifold learning

Tomaremos un desvío de modelos generativos para entender un componente importante. El término *manifold learning* se utiliza para técnicas de reducción de dimensionalidad *no lineales* que parten de PCA y van *más allá*. 

Vamos a revisar detalles importantes del funcionamiento de PCA que nos servirán más adelante. 

Primero, el dataset:

In [ ]:
# Descargamos el dataset de los pingüinos
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd

penguins = fetch_openml('penguins')

# Suprimimos la columna 'island' y las filas con NaNs
X = penguins['data'][:,1:6]
lab = penguins['target'][~np.any(np.isnan(X), axis=1)]
X = X[~np.any(np.isnan(X), axis=1)]
print(penguins['feature_names'])
print(X.shape)
pd.DataFrame(X, columns=penguins['feature_names'][1:6])

## Revisión de PCA

La idea de PCA es encontrar una proyección de un dataset $\mathbf{X}$ con $n$ filas y $p$ columnas, hacia un dataset *de componentes principales* $\mathbf{P}$, también de $n$ filas y $p$ columnas. Esta transformación es lineal: $\mathbf{P} = \mathbf{XR}$ siendo $\mathbf{R}$ la denominada *matriz de rotación* o *de ejes principales*, con $p$ filas y $p$ columnas. "Entrenar" PCA consiste en conseguir los valores de $\mathbf{R}$.



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Entrenamos PCA sin reducir
X_s = StandardScaler().fit_transform(X)
pca = PCA()
pca.fit(X_s)

# Obtenemos R y P
R = pca.components_
P = pca.transform(X_s)
print(R.shape)
print(P.shape)

La matriz del dataset proyectado $\mathbf{P}$ tiene varias propiedades importantes. Cada columna $\mathbf{P}_{*,j}$ es el resultado de una combinación lineal de todas las columnas de $\mathbf{X}$ multiplicadas por la columna de rotación correspondiente $\mathbf{R}_{*,j}$ y las columnas en $\mathbf{P}$ están descorrelacionadas. 

In [ ]:
pd.DataFrame(pca.explained_variance_)
pd.DataFrame(np.round(np.corrcoef(P.T), 10))

Lo más importante es que las varianzas de las columnas están ordenadas, de modo que la primera dimensión concentra la mayor parte de la varianza en el dataset, y así sucesivamente.

In [ ]:
print(np.diag(np.cov(P.T)))
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

De modo que, si reducimos nuestros dataset a $p=2$ dimensiones, conservaríamos el 84.6% de la varianza del dataset

In [ ]:
np.sum(pca.explained_variance_ratio_[0:2])

Intentemos graficar la reducción a dos dimensiones:

In [ ]:
import matplotlib.pyplot as plt
colors=['red' if x=='Adelie' else 'green' if x=='Chinstrap' else 'black' for x in lab]
plt.scatter(P[:,0], P[:,1], c=colors)
plt.show()

## Métodos no-lineales

PCA es lineal, en el sentido en que la transformación de $\mathbf{X}$ a $\mathbf{P}$ es una función lineal. Existe una gran cantidad de métodos no lineales para reducción de dimensionalidad, con distintos propósitos.

Uno de los más útiles es t-SNE (t-Stochastic Nearest Neighbors Embedding), que es muy útil para visualización.

In [ ]:
from sklearn.manifold import TSNE
T = TSNE(n_components=2).fit_transform(X_s)
plt.scatter(T[:,0], T[:,1], c=colors)
plt.show()